# Consultas Integradas

Este notebook tiene el fin de generar las consultas planteadas en el trabajo práctico integrador

#### Importo librerias

In [1]:
# Limpia todas las variables existentes
%reset -f

# Habilita autoreload para recargar automáticamente todos los módulos
%load_ext autoreload
%autoreload 2

In [25]:
import os
from pathlib import Path
import pandas as pd
from pymongo.errors import ConnectionFailure
from db_connections import client, db_neo4j, db_redis
from src import mongo, neo4j, utils, redis
from IPython.display import display

In [3]:
# =====================
# TEST DE CONEXIONES
# =====================

#Conecto con MongoDB
try:
    client.admin.command("ping")  # fuerza conexión al servidor
    print("✅ Conexión a MongoDB verificada.")
except ConnectionFailure as e:
    print(f"❌ Falló la conexión: {type(e).__name__} - {e}")

#Conecto con Neo4j
try:
    db_neo4j.verify_connectivity()
    print("✅ Conexión a Neo4j verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

#Conecto con Redis
try:
    db_redis.ping()
    print("✅ Conexión a Redis verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

✅ Conexión a MongoDB verificada.
✅ Conexión a Neo4j verificada.
✅ Conexión a Redis verificada.


### A.Mostrar los usuarios que visitaron “Bariloche”. 

In [4]:
query = """ 
MATCH (U:Usuario)-[:VISITO]->(D:Destino)
WHERE D.ciudad='La Plata'
RETURN U.nombre AS Nombre, U.apellido AS Apellido
"""

usuarios = neo4j.consulta(db_neo4j, query)
usuarios

print("\n" + "-"*60)
print("USUARIOS QUE VISITARON BARILOCHE")
print("-"*60)
display(usuarios.style.hide(axis="index"))


------------------------------------------------------------
USUARIOS QUE VISITARON BARILOCHE
------------------------------------------------------------


Nombre,Apellido
Agapito,Sedano
Candela,Solano
Ovidio,Romero
Fernanda,Romero
Eliseo,Sobrino


### B.Mostrar los amigos de Juan que visitaron algún destino que visitó él, mostrar el nombre del Usuario y el destino. 

In [5]:
import pandas as pd

# Pedimos el nombre del usuario
nombre_usuario = input("Introduce tu nombre para saber qué lugares visitaste con tus amigos: ")


query = """
MATCH (u:Usuario)
WHERE toLower(u.nombre) CONTAINS toLower($nombre)
MATCH (u)-[:AMIGO_DE]-(amigo:Usuario)
MATCH (u)-[:VISITO]->(d:Destino)<-[:VISITO]-(amigo)
RETURN 
    amigo.nombre AS Nombre,
    collect(DISTINCT d.ciudad) AS Destinos_Compartidos
ORDER BY Nombre
"""

#Encabezado
print("\n" + "-"*60)
print(f"{nombre_usuario.upper()} TUS AMIGOS VISITARON ESTOS MISMOS DESTINOS QUE TÚ")
print("-"*60 + "\n")

# Ejecutamos la consulta usando la función consulta() que devuelve un DataFrame
usuarios = neo4j.consulta(db_neo4j, query, parametros={"nombre": nombre_usuario})

# Convertimos listas a texto legible
usuarios["Destinos_Compartidos"] = usuarios["Destinos_Compartidos"].apply(
    lambda lista: ", ".join(lista) if isinstance(lista, list) else lista
)



# Muestro solo cidades

# Extraer ciudades únicas
ciudades_unicas = set()
usuarios['Destinos_Compartidos'].str.split(', ').apply(ciudades_unicas.update)

# Convertir a lista y ordenar
ciudades_unicas = sorted(list(ciudades_unicas))

# Convertir a DataFrame para poder usar display y style
df_ciudades = pd.DataFrame(ciudades_unicas, columns=["Ciudad"])
display(df_ciudades.style.hide(axis="index"))


print("En detalle se tiene que ")

# Mostramos el resultado
if usuarios.empty:
    print("No se encontraron amigos que hayan visitado los mismos destinos.")
else:
    display(usuarios.style.hide(axis="index"))




Introduce tu nombre para saber qué lugares visitaste con tus amigos:  j



------------------------------------------------------------
J TUS AMIGOS VISITARON ESTOS MISMOS DESTINOS QUE TÚ
------------------------------------------------------------



Ciudad
Bariloche
CABA
Chilecito
Concordia
Córdoba
El Calafate
Formosa
Iguazú
Neuquén
Paraná


En detalle se tiene que 


Nombre,Destinos_Compartidos
Adela,Posadas
Adrián,"Córdoba, Villa Carlos Paz, Paraná, Formosa, San Martín de los Andes, Bariloche"
Amor,"San Fernando, Villa Carlos Paz, Posadas, Salta, Formosa"
Ariadna,"Resistencia, Posadas, Salta, San Luis, Santa Fe, Santiago del Estero"
Calista,"Córdoba, San Martín de los Andes, Bariloche"
Ciro,"Resistencia, Paraná, Concordia"
Coral,"Río Cuarto, Chilecito, Rosario"
Dani,"Posadas, Neuquén, Santiago del Estero"
David,"Río Cuarto, Concordia, Santa Rosa, Posadas, Salta, San Luis"
Emiliana,"Puerto Madryn, Posadas, El Calafate, Santiago del Estero, San Salvador de Jujuy, Salta"


### C. Sugerir destinos a un usuario que no haya visitado él ni sus amigos. 

In [6]:
nombre_usuario = input("Introduce tu nombre para saber qué lugares nuevos que no visitaste ni tu ni tus amigos: ")

query = """
MATCH (u:Usuario)
WHERE toLower(u.nombre) CONTAINS toLower($nombre)
MATCH (d:Destino)
WHERE 
  NOT EXISTS { MATCH (d)<-[:VISITO]-(u) } AND
  NOT EXISTS { MATCH (d)<-[:VISITO]-(u)-[:AMIGO_DE]-(amigo) }
RETURN DISTINCT d.ciudad AS Destinos_No_Visitados
ORDER BY d.ciudad
"""

print("\n" + "-"*60)
print(f"DESTINOS NUEVOS PARA {nombre_usuario.upper()} Y SUS AMIGOS")
print("-"*60 + "\n")

destinos = neo4j.consulta(db_neo4j, query, parametros={"nombre": nombre_usuario})

if destinos.empty:
    print("No hay destinos nuevos disponibles.")
else:
    # Mostrar cada destino en lista
     display(destinos.style.hide(axis="index"))




Introduce tu nombre para saber qué lugares nuevos que no visitaste ni tu ni tus amigos:  j



------------------------------------------------------------
DESTINOS NUEVOS PARA J Y SUS AMIGOS
------------------------------------------------------------



Destinos_No_Visitados
Bariloche
CABA
Chilecito
Concordia
Corrientes
Córdoba
El Calafate
Formosa
Gualeguaychú
Iguazú


### d. Recomendar destinos basados en viajes de amigos.

In [7]:
nombre_usuario = input("Introduce tu nombre para saber qué lugares te recomendamos: ")

query = """
MATCH (u:Usuario)
WHERE toLower(u.nombre) CONTAINS toLower($nombre)
MATCH (u)-[:AMIGO_DE]-(amigo:Usuario)
MATCH (amigo)-[:VISITO]->(d:Destino)
WHERE NOT EXISTS { MATCH (u)-[:VISITO]->(d) } 
RETURN DISTINCT d.ciudad AS Destino_Recomendado
ORDER BY d.ciudad
"""

print("\n" + "-"*60)
print(f"RECOMENDACIONES PARA {nombre_usuario.upper()} EN FUNCIÓN DE SUS AMIGOS")
print("-"*60 + "\n")

destinos = neo4j.consulta(db_neo4j, query, parametros={"nombre": nombre_usuario})

if destinos.empty:
    print("No hay destinos recomendados nuevos para vos.")
else:
    display(destinos.style.hide(axis="index"))


Introduce tu nombre para saber qué lugares te recomendamos:  j



------------------------------------------------------------
RECOMENDACIONES PARA J EN FUNCIÓN DE SUS AMIGOS
------------------------------------------------------------



Destino_Recomendado
Bariloche
CABA
Chilecito
Concordia
Corrientes
Córdoba
El Calafate
Formosa
Gualeguaychú
Iguazú


### e. Listar los hoteles en los destinos recomendados del punto anterior. 

In [14]:
nombre_base = "clase"
coleccion = "hoteles"

# Lista de destinos recomendados
lista_destinos = destinos["Destino_Recomendado"].dropna().unique().tolist()

if not lista_destinos:
    print("No hay destinos recomendados disponibles.")
else:
    filtro = {"ciudad": {"$in": lista_destinos}}
    proyeccion = {"_id": 0, "nombre": 1, "ciudad": 1, "direccion": 1}

    # Obtener datos de Mongo y convertir a DataFrame
    cursor = mongo.obtener_cursor(
        nombre_base=nombre_base,
        nombre_coleccion=coleccion,
        filtro=filtro,
        proyeccion=proyeccion
    )

    hoteles = pd.DataFrame(list(cursor))

    if hoteles.empty:
        print("No se encontraron hoteles en los destinos recomendados.")
    else:
        # Ordenar por ciudad y luego por nombre de hotel
        display(hoteles.style.hide(axis="index"))
       



nombre,ciudad
Hermanos Soria S.Coop. Hotel,La Plata
Espinosa & Asociados S.C.P Hotel,La Plata
Danilo Bas Pérez S.Coop. Hotel,Mar del Plata
Grupo Company S.L.L. Hotel,Mar del Plata
Compañía Hurtado & Asociados S.Coop. Hotel,Mar del Plata
Gonzalez y Yáñez S.Com. Hotel,CABA
Manuel Romeu Mayoral S.Coop. Hotel,CABA
Familia Galiano S.L. Hotel,San Fernando
Compañía Crespo & Asociados S.A. Hotel,San Fernando
Distribuciones VJ S.Coop. Hotel,San Fernando


### f. Ver las reservas en proceso, es decir que aún no están concretadas. 

In [15]:
claves = db_redis.keys("reserva_temp:*")
print(f"Cantidad de reservas en proceso {len(claves)}\n")

if claves:
    cantidad = int(input ("¿Cuántas se desean listar?"))
    print(f"Se imprimen las primeras {cantidad}:")
    for clave in claves[:cantidad]:
        datos = db_redis.hgetall(clave)
        tiempo_restante = db_redis.ttl(clave)
        print(f"{clave}: {datos} | TTL: {tiempo_restante} segundos")

Cantidad de reservas en proceso 0



### g. Listar los usuarios conectados actualmente. 

In [16]:
claves = db_redis.keys("usuario:*:sesion")
print(f"Cantidad de usuarios conectados {len(claves)}\n")

if claves:
    print("Usuarios:")
    for clave in claves:
        usuario_id = clave.split(":")[1]
        estado = db_redis.get(clave)
        tiempo_restante = db_redis.ttl(clave)
        print(f"Usuario {usuario_id} → sesión: {estado} | TTL: {tiempo_restante} segundos")

Cantidad de usuarios conectados 0



### h. Mostrar los destinos con precio inferior a $100.000

In [17]:
precio=100000
base = "clase"
coleccion = "destinos"
filtro={"precio_promedio": {"$lt": precio}}
proyeccion = {"_id":0}

# Busca en caché y sino consulta en MongoDB
resultado = redis.obtener_cache(coleccion, filtro)
if resultado is None:
    print("Consulta hecha en Mongo")
    cursor = mongo.obtener_cursor(base,coleccion,filtro=filtro,proyeccion=proyeccion)
    resultado = list(cursor)
    if resultado:
        redis.guardar_en_cache(coleccion,filtro,resultado,ttl=300)
else:
    print("Consulta hecha en Redis")
        
df = pd.DataFrame(resultado)
if not df.empty:
    df_ordenado = df.sort_values(by="destino_id")
    display(df_ordenado.style.hide(axis="index"))
else:
    print(f"No existen destinos con precio inferior a {precio}")

Consulta hecha en Redis


destino_id,provincia,ciudad,pais,tipo,precio_promedio
1,Buenos Aires,La Plata,Argentina,Cultural,56556
3,Ciudad Autónoma de Buenos Aires,CABA,Argentina,Playa,86579
6,Chubut,Puerto Madryn,Argentina,Cultural,57811
9,Córdoba,Río Cuarto,Argentina,Relax,56956
14,Formosa,Formosa,Argentina,Cultural,91853
16,La Pampa,Santa Rosa,Argentina,Montaña,90758
18,La Rioja,Chilecito,Argentina,Cultural,74312
19,Mendoza,Mendoza,Argentina,Aventura,75353
23,Neuquén,Neuquén,Argentina,Relax,82722
24,Neuquén,San Martín de los Andes,Argentina,Aventura,70656


## i. Mostrar todos los Hoteles de “Jujuy”.

In [18]:
nombre_base = "clase"
coleccion = "hoteles"
ciudad = "San Salvador de Jujuy"
filtro = {"ciudad": ciudad}
proyeccion = {"_id": 0, "nombre": 1, "ciudad": 1, "direccion": 1}

# Busca en caché y sino consulta en MongoDB
resultado = redis.obtener_cache(coleccion, filtro)
if resultado is None:
    print("Consulta hecha en Mongo")
    cursor = mongo.obtener_cursor(nombre_base=nombre_base,nombre_coleccion=coleccion,filtro=filtro,proyeccion=proyeccion)
    resultado = list(cursor)
    if resultado:
        redis.guardar_en_cache(coleccion,filtro,resultado,ttl=300)
else:
    print("Consulta hecha en Redis")

df_hoteles = pd.DataFrame(resultado)

if not df_hoteles.empty:
    df_ordenado = df_hoteles.sort_values(by="nombre")
    display(df_ordenado.style.hide(axis="index"))
else:
     print("No se encontraron hoteles en los destinos recomendados.")       

Consulta hecha en Redis


nombre,ciudad
Comercializadora Soria S.L. Hotel,San Salvador de Jujuy
Familia Guitart S.L. Hotel,San Salvador de Jujuy
Hotel Botella y asociados S.L.U. Hotel,San Salvador de Jujuy
Manufacturas Puga & Asociados S.L.L. Hotel,San Salvador de Jujuy
Moliner y Torrents S.L.N.E Hotel,San Salvador de Jujuy


## j. Mostrar la cantidad de hoteles de un destino que guste. 

In [19]:
nombre_base = "clase"
coleccion = "destinos"
db = client[nombre_base]
ciudad = input ("Ingrese una ciudad que le guste")
filtro={"ciudad":ciudad}

resultado = redis.obtener_cache(coleccion, filtro)
if resultado is None:
    print("Consulta hecha en Mongo")
    resultado = db[coleccion].count_documents(filtro)
    if resultado:
        redis.guardar_en_cache(coleccion,filtro,resultado,ttl=300)
else:
    print("Consulta hecha en Redis")
    
print(f"La cantidad de hoteles en {ciudad} son: {resultado}") 



""" 
Consultar a Dario, lo habia hecho dinamico para que pudiera escoger entre ciudad y provincia,
 tambien el contador(funcion que cree, es dinamca, se contruye en funcion de la necesidad), aca le agregue la cantidad por ciudad o prov

## j. Mostrar la cantidad de hoteles de un destino que guste. 
agrupar= input("Introduce provincia o ciudad")
lugar= input("Introduce la provincia o ciudad ")

cursor = mongo.contador(
    nombre_base="clase",
    coleccion="hoteles",
    agrupacion=agrupar,
    campo_calculo="hotel_id",
#    filtrar={"provincia": provincia_elegida}
)


hoteles = pd.DataFrame(cursor)
hoteles
"""

Ingrese una ciudad que le guste La Plata


Consulta hecha en Mongo
La cantidad de hoteles en La Plata son: 1


' \nConsultar a Dario, lo habia hecho dinamico para que pudiera escoger entre ciudad y provincia,\n tambien el contador(funcion que cree, es dinamca, se contruye en funcion de la necesidad), aca le agregue la cantidad por ciudad o prov\n\n## j. Mostrar la cantidad de hoteles de un destino que guste. \nagrupar= input("Introduce provincia o ciudad")\nlugar= input("Introduce la provincia o ciudad ")\n\ncursor = mongo.contador(\n    nombre_base="clase",\n    coleccion="hoteles",\n    agrupacion=agrupar,\n    campo_calculo="hotel_id",\n#    filtrar={"provincia": provincia_elegida}\n)\n\n\nhoteles = pd.DataFrame(cursor)\nhoteles\n'

### k. Mostrar las actividades de “Ushuaia” del tipo “aventura”.

In [20]:
nombre_base = "clase"
coleccion = "actividades"
tipo="aventura"
ciudad="Ushuaia"
filtro = {"ciudad":ciudad,"tipo":tipo}
proyeccion = {"_id":0}

resultado = redis.obtener_cache(coleccion, filtro)
if resultado is None:
    cursor = mongo.obtener_cursor(base,coleccion,filtro=filtro,proyeccion=proyeccion)
    print("Consulta hecha en Mongo")
    resultado = list(cursor)
    if resultado:
        redis.guardar_en_cache(coleccion,filtro,resultado,ttl=300)
else:
    print("Consulta hecha en Redis")
    
df = pd.DataFrame(resultado)
if not df.empty:
    df_ordenado = df.sort_values(by="actividad_id")
    display(df_ordenado.style.hide(axis="index"))
else:
    print(f"No existen actividades de tipo {tipo} en {ciudad}")

Consulta hecha en Mongo
No existen actividades de tipo aventura en Ushuaia


### l. Mostrar la cantidad de reservas concretadas de cada usuario. Mostrar el usuario y la cantidad 

In [ ]:
import pandas as pd

nombre_base = "clase"
coleccion = "reservas"
db = client[nombre_base]

# Filtro: solo reservas confirmadas o pagadas
estados = ["Confirmada", "Pagada"]
filtro = {"estado": {"$in": estados}}

# Pipeline de agregación
pipeline = [
    {"$match": filtro},
    
    # Agrupar por destino_id y contar reservas
    {"$group": {
        "_id": "$destino_id",
        "Reservas_concretadas": {"$sum": 1}
    }},
    
    # Unir con la colección destinos para obtener el nombre del destino
    {"$lookup": {
        "from": "destinos",
        "localField": "_id",
        "foreignField": "destino_id",
        "as": "destino_info"
    }},
    
    # Desanidar el array de destino_info
    {"$unwind": "$destino_info"},
    
    # Seleccionar campos que queremos mostrar
    {"$project": {
        "_id": 0,
        "Destino": "$destino_info.nombre",
        "Reservas_concretadas": 1
    }},
    
    # Orden descendente por cantidad de reservas
    {"$sort": {"Reservas_concretadas": -1}},
    
    # Limitar a 1 si queremos solo el más visitado (opcional)
    {"$limit": 1}
]

# Comprobar cache en Redis
resultado = redis.obtener_cache(coleccion, filtro)
if resultado is None:
    print("Consulta hecha en Mongo")
    cursor = db[coleccion].aggregate(pipeline)
    resultado = list(cursor)
    if resultado:
        redis.guardar_en_cache(coleccion, filtro, resultado, ttl=300)
else:
    print("Consulta hecha en Redis")

# Convertir a DataFrame y mostrar
df = pd.DataFrame(resultado)
df = df[["Destino", "Reservas_concretadas"]]
display(df.style.hide(axis="index"))


#### Estadísticas

###### i.Destino más visitado

In [38]:
import pandas as pd

# Configuración de base y colección
nombre_base = "clase"
coleccion = "reservas"
db = client[nombre_base]

# Filtro: solo reservas confirmadas o pagadas
estados = ["Confirmada", "Pagada"]
filtro = {"estado": {"$in": estados}}

# Pipeline de agregación corregido
pipeline = [
    # Filtrar reservas válidas
    {"$match": filtro},

    # Agrupar por destino_id y contar reservas
    {"$group": {
        "_id": "$destino_id",
        "Reservas_concretadas": {"$sum": 1}
    }},

    # Unir con la colección destinos para obtener el nombre del destino
    {"$lookup": {
        "from": "destinos",
        "localField": "_id",
        "foreignField": "destino_id",
        "as": "destino_info"
    }},

    # Desanidar el array para acceder al nombre
    {"$unwind": "$destino_info"},

    # Seleccionar solo los campos que nos interesan
    {"$project": {
        "_id": 0,
        "Destino": "$destino_info.nombre",
        "Reservas_concretadas": 1
    }},

    # Ordenar de mayor a menor reservas
    {"$sort": {"Reservas_concretadas": -1}}
]

# Ejecutar agregación
cursor = db[coleccion].aggregate(pipeline)
resultado = list(cursor)

# Mostrar resultados
df = pd.DataFrame(resultado)
display(df)


,Reservas_concretadas
0,51
1,46
2,44
3,40
4,39
5,39
6,38
7,37
8,36
9,34


###### ii.Hotel más barato

###### iii.Actividad más popular.

#### Modificaciones

###### a.Incrementar el precio de las actividades de Tucuman en 5% 

###### b. Agregar al hotel id=1 el servicio de SPA 

###### c. Eliminar el destino que desee

###### d. Eliminar un usuario que desee 

###### e. Eliminar las relaciones AMIGO_DE para un usuario que quiera. 